# EUROPEAN SOCCER PLAYERS DATA ANALYSIS

Click [here](https://www.kaggle.com/datasets/hugomathien/soccer) for a detail report from *Kaggle* on the European Soccer Dataset.

In [1]:
import os
import sqlite3
import pandas as pd
import opendatasets as od
from datetime import date

In [2]:
# Create a SQL connection
con = sqlite3.connect("soccer\database.sqlite")

# Query to list all tables
sql_query = """SELECT name FROM sqlite_master WHERE type = 'table';"""

cur = con.cursor()
cur.execute(sql_query)
print("List of all tables in the database")
print(cur.fetchall())
con.close()

List of all tables in the database
[('sqlite_sequence',), ('Player_Attributes',), ('Player',), ('Match',), ('League',), ('Country',), ('Team',), ('Team_Attributes',)]


In [44]:
tables = {'sqlite_sequence': 'df_seq', 'Player_Attributes': 'df_pla_attr', 'Player': 'df_pla', 
          'Match': 'df_match', 'League': 'df_leag', 'Country': 'df_coun', 'Team': 'df_team', 
          'Team_Attributes': 'df_team_attrs'}

folder = 'data'
for table in os.listdir(folder):
    df = pd.read_csv('data/{}' .format(table))
    globals()[tables.get(str(table)[:-4])] = df

### Players and their Attributes

In [48]:
# Sort date by descending to have recent entry at the top
df_pla_attr.sort_values(by='date', ascending = False)

# Remove duplicates whiles keeping the recent entry of player's attribute
df_pla_attr.drop_duplicates(subset='player_fifa_api_id', inplace=True)

In [49]:
# Print the number of samples and columns in the dataset
print('The number of players in the dataset is', df_pla_attr.shape[0], 
     '\nand the number of columns is', df_pla_attr.shape[1])

The number of players in the dataset is 11062 
and the number of columns is 42


In [50]:
# merge two datasets, Player and Player_attributes
df_players = pd.merge(df_pla, df_pla_attr, on='player_api_id')

In [51]:
# Drop columns 
columns = ['id_x', 'id_y', 'player_fifa_api_id_y', 'date','crossing', 'finishing', 'heading_accuracy',
       'short_passing', 'volleys', 'dribbling', 'curve', 'free_kick_accuracy', 'long_passing', 
       'acceleration', 'sprint_speed', 'agility', 'reactions', 'balance', 'shot_power', 'jumping', 
       'stamina', 'strength', 'long_shots', 'aggression', 'interceptions', 'positioning',
       'vision', 'penalties', 'marking', 'standing_tackle', 'sliding_tackle',
       'gk_diving', 'gk_handling', 'gk_kicking', 'gk_positioning', 'gk_reflexes']

df_players.drop(columns=columns, inplace=True)

In [52]:
# Calculate Age from birthday
today = date.today()

df_players['birthday'] = pd.to_datetime(df_players['birthday']).dt.date
df_players['Age'] = today - df_players['birthday']
df_players['Age'] = df_players['Age'].dt.days.astype("int16") // 365

In [53]:
# Calculate the BMI
#convert weight pounds to kg
wei_kg = df_players['weight'] * 0.45359237

df_players['BMI'] = round((wei_kg / df_players['height'] / df_players['height']) * 10000, 2)

In [54]:
df_players.head()

,player_api_id,player_name,player_fifa_api_id_x,birthday,height,weight,overall_rating,potential,preferred_foot,attacking_work_rate,defensive_work_rate,ball_control,Age,BMI
0,505942,Aaron Appindangoye,218353,1992-02-29,182.88,187,67.0,71.0,right,medium,medium,49.0,32,25.36
1,155782,Aaron Cresswell,189615,1989-12-15,170.18,146,74.0,76.0,left,high,medium,71.0,34,22.87
2,162549,Aaron Doran,186170,1991-05-13,170.18,163,65.0,67.0,right,medium,medium,67.0,33,25.53
3,30572,Aaron Galindo,140161,1982-05-08,182.88,198,69.0,69.0,right,medium,medium,62.0,42,26.85
4,23780,Aaron Hughes,17725,1979-11-08,182.88,154,70.0,70.0,right,medium,medium,58.0,44,20.89


In [55]:
df_players.describe()

,player_api_id,player_fifa_api_id_x,height,weight,overall_rating,potential,ball_control,Age,BMI
count,11062.000000,11062.000000,11062.000000,11062.000000,11060.000000,11060.000000,11060.000000,11062.000000,11062.000000
mean,156569.025493,165654.051799,181.868317,168.385102,67.961935,71.037432,62.099005,36.894233,23.066492
std,160704.573209,58652.843912,6.367879,14.988696,6.316694,6.214862,15.309941,5.460812,1.316831
min,2625.000000,2.000000,157.480000,117.000000,42.000000,49.000000,9.000000,25.000000,17.590000
25%,35554.500000,151874.500000,177.800000,159.000000,64.000000,67.000000,58.000000,33.000000,22.170000
50%,96619.500000,184656.000000,182.880000,168.000000,68.000000,71.000000,65.000000,36.000000,23.060000
75%,212460.750000,203882.000000,185.420000,179.000000,72.000000,75.000000,72.000000,41.000000,23.870000
max,750584.000000,234141.000000,208.280000,243.000000,94.000000,94.000000,96.000000,57.000000,30.870000


In [56]:
# data only on RIGHT preferred foot
df_players[df_players['preferred_foot'] == 'right'].describe()

,player_api_id,player_fifa_api_id_x,height,weight,overall_rating,potential,ball_control,Age,BMI
count,8373.000000,8373.000000,8373.000000,8373.000000,8373.000000,8373.000000,8373.000000,8373.000000,8373.000000
mean,155603.275290,165483.439150,182.101285,168.959393,67.942434,71.015407,61.425296,36.948764,23.086213
std,160825.374993,58758.997623,6.401550,15.023060,6.372629,6.212234,15.891784,5.495655,1.317375
min,2625.000000,6.000000,157.480000,117.000000,42.000000,49.000000,9.000000,25.000000,17.590000
25%,35417.000000,150594.000000,177.800000,159.000000,64.000000,67.000000,57.000000,33.000000,22.180000
50%,95336.000000,184484.000000,182.880000,168.000000,68.000000,71.000000,65.000000,37.000000,23.100000
75%,210691.000000,203863.000000,187.960000,179.000000,72.000000,75.000000,72.000000,41.000000,23.870000
max,750584.000000,234141.000000,208.280000,243.000000,93.000000,94.000000,93.000000,57.000000,30.870000


In [57]:
# data only on LEFT preferred foot
df_players[df_players['preferred_foot'] == 'left'].describe()

,player_api_id,player_fifa_api_id_x,height,weight,overall_rating,potential,ball_control,Age,BMI
count,2687.000000,2687.000000,2687.000000,2687.000000,2687.000000,2687.000000,2687.000000,2687.000000,2687.000000
mean,159581.794194,166198.499070,181.135936,166.588016,68.022702,71.106066,64.198362,36.726089,23.005601
std,160375.914006,58348.460408,6.203949,14.743058,6.139894,6.223702,13.117620,5.349943,1.313817
min,2768.000000,2.000000,160.020000,121.000000,46.000000,51.000000,11.000000,25.000000,18.410000
25%,36556.500000,154346.000000,177.800000,157.000000,64.000000,67.000000,60.000000,33.000000,22.150000
50%,103139.000000,185323.000000,180.340000,165.000000,68.000000,71.000000,66.000000,36.000000,23.060000
75%,215416.500000,203950.500000,185.420000,176.000000,72.000000,75.000000,72.000000,41.000000,23.870000
max,744907.000000,233911.000000,203.200000,225.000000,94.000000,94.000000,96.000000,54.000000,29.570000


In [58]:
df_players.to_csv('data/Players_and_thier_Attributes.csv', index=False)

### Teams and their attributes

In [86]:
df_team_attr = df_team_attrs.copy()

In [87]:
df_team_attr.head()

,id,team_fifa_api_id,team_api_id,date,buildUpPlaySpeed,buildUpPlaySpeedClass,buildUpPlayDribbling,buildUpPlayDribblingClass,buildUpPlayPassing,buildUpPlayPassingClass,...,chanceCreationShooting,chanceCreationShootingClass,chanceCreationPositioningClass,defencePressure,defencePressureClass,defenceAggression,defenceAggressionClass,defenceTeamWidth,defenceTeamWidthClass,defenceDefenderLineClass
0,1,434,9930,2010-02-22 00:00:00,60,Balanced,NaN,Little,50,Mixed,...,55,Normal,Organised,50,Medium,55,Press,45,Normal,Cover
1,2,434,9930,2014-09-19 00:00:00,52,Balanced,48.0,Normal,56,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
2,3,434,9930,2015-09-10 00:00:00,47,Balanced,41.0,Normal,54,Mixed,...,64,Normal,Organised,47,Medium,44,Press,54,Normal,Cover
3,4,77,8485,2010-02-22 00:00:00,70,Fast,NaN,Little,70,Long,...,70,Lots,Organised,60,Medium,70,Double,70,Wide,Cover
4,5,77,8485,2011-02-22 00:00:00,47,Balanced,NaN,Little,52,Mixed,...,52,Normal,Organised,47,Medium,47,Press,52,Normal,Cover


In [88]:
# Sort date by descending to have recent entry at the top
df_team_attr.sort_values(by='date', ascending = False)

# Remove duplicates whiles keeping the recent entry of team's attribute
df_team_attr.drop_duplicates(subset='team_fifa_api_id', inplace=True)

In [89]:
# Print the number of samples and columns in the dataset
print('The number of teams in the dataset is', df_team_attr.shape[0], 
     '\nand the number of columns is', df_team_attr.shape[1])

The number of teams in the dataset is 285 
and the number of columns is 25


In [90]:
# merge two datasets, Team and Team_attributes
df_teams = pd.merge(df_team, df_team_attr, on='team_api_id')

In [93]:
print(df_teams.columns.tolist())

['team_api_id', 'team_fifa_api_id_x', 'team_long_name', 'team_short_name', 'buildUpPlaySpeedClass', 'buildUpPlayPassingClass', 'buildUpPlayPositioningClass', 'chanceCreationPassingClass', 'chanceCreationCrossingClass', 'chanceCreationShootingClass', 'chanceCreationPositioningClass', 'defencePressureClass', 'defenceAggressionClass', 'defenceTeamWidthClass', 'defenceDefenderLineClass']


In [92]:
# Drop columns 
columns = ['id_x', 'id_y', 'team_fifa_api_id_y', 'date', 'buildUpPlaySpeed', 'buildUpPlayDribbling', 
           'buildUpPlayDribblingClass', 'buildUpPlayPassing', 'chanceCreationPassing', 'chanceCreationCrossing',  
           'chanceCreationShooting', 'defencePressure', 'defenceAggression', 'defenceTeamWidth']


df_teams.drop(columns=columns, inplace=True)

In [81]:
df_teams['buildUpPlaySpeed'].value_counts()

30    60
70    49
45    23
35    21
55    20
65    20
50    20
60    16
40    13
64     4
53     4
54     3
48     3
66     3
47     3
62     2
67     2
41     2
63     2
32     2
52     2
59     1
61     1
73     1
39     1
58     1
46     1
38     1
69     1
36     1
42     1
51     1
Name: buildUpPlaySpeed, dtype: int64